In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer,  AutoModelWithLMHead, get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from tqdm import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
root = ''
last_checkpoint = root+'generation/rusmall.pt'
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
# model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2').to(device)
model_jokes = torch.load(last_checkpoint)
special_tokens = {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '[MASK]'}
tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


3

In [3]:
class Jokes(Dataset):

    def __init__(self, csv_file, tokenizer):
        data = pd.read_csv(csv_file)
        data = data[data['label'] == 1]
        data = data.drop(columns=['label'])
        data['text'] = data['text'].apply(lambda x: x.replace('\n', ''))
        data['text'] = data['text'].apply(lambda x: x.replace(u'\xa0', ''))
        data = data.reset_index(drop=True)
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # text = '<bos> ' + self.data.at[idx, 'text'] + ' <eos>'
#         text = self.tokenizer.encode('<bo> ' + self.data.at[idx, 'text'] + ' <jokee>')
        text = [tokenizer.bos_token_id] + self.tokenizer.encode(self.data.at[idx, 'text']) + [tokenizer.eos_token_id]
        
        return torch.tensor(text, dtype=torch.long)

def my_collate_fn(batch):
    return pad_sequence(batch, batch_first=True)

In [4]:
train_dataset = Jokes(root+'train_full.csv', tokenizer)
# train_sampler = DistributedSampler(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [5]:
def mask_tokens(inputs: torch.Tensor, tokenizer):
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, 0.15)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

In [6]:
num_epochs = 2
gradient_accumulation_steps = 1250
max_grad_norm = 0.5
logging_steps = 5
t_total = len(train_loader) // gradient_accumulation_steps * num_epochs
warmup_steps = 5
print(f't_total is {t_total}')

t_total is 136


In [7]:
optimizer =  torch.optim.Adam(model_jokes.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)

In [8]:
def generate(input_text, model, tokenizer):
    input_text = '<bos>' + input_text 
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    # input_ids = torch.tensor([[tokenizer.bos_token_id] + tokenizer.encode(input_text)]).to(device)
    text = model.generate(
            input_ids=input_ids,
            repetition_penalty=5.0,
            top_k=7, 
            top_p=0.92, 
            max_length=32,
            num_beams=7,
            no_repeat_ngram_size=8,
            pad_token_id = tokenizer.pad_token_id,
            bos_token_id = tokenizer.bos_token_id,
            eos_token_id = tokenizer.eos_token_id,
#             early_stopping=True,
            do_sample=True,
            length_penalty=1.5,
#             num_return_sequences=5,
            )
    return tokenizer.decode(text[0], skip_special_tokens=True)
#     return [tokenizer.decode(t) for t in text]

In [9]:
model_jokes.zero_grad()
cudas = []
global_step = 0
tr_loss, logging_loss = 0.0, 0.0
for epoch in trange(num_epochs, position=0, leave=True, desc='Epochs'):
    epoch_iterator = tqdm(train_loader, desc="Training", position=0, leave=True)
    for step, batch in enumerate(epoch_iterator):
#         inputs, labels = mask_tokens(batch, tokenizer)
        inputs = batch.to(device)
        labels = batch.to(device)
        model_jokes.train()
        outputs = model_jokes(inputs, labels=labels)
        loss = outputs[0]
        loss = loss / gradient_accumulation_steps
        loss.backward()
        tr_loss += loss.item()

        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model_jokes.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            model_jokes.zero_grad()
            global_step += 1
            cudas.append(torch.cuda.memory_reserved(0) / (2 ** 30))

            if global_step % logging_steps == 0:
                print(generate('Женщина должна быть', model_jokes, tokenizer))
                print(f'==> Train Loss {(tr_loss - logging_loss) / logging_steps} at {global_step} step')

                logging_loss = tr_loss

Training:   7%|▋         | 6253/85316 [04:50<1:26:22, 15.26it/s]

Женщина должна быть такой, какой она есть.
==> Train Loss 3.6829359419178216 at 5 step


Training:  15%|█▍        | 12503/85316 [09:40<1:26:57, 13.96it/s]

Женщина должна быть не только красивой, но и умной.
==> Train Loss 3.680321445898153 at 10 step


Training:  22%|██▏       | 18752/85316 [14:31<1:34:31, 11.74it/s]

...

Женщина должна быть женщиной, а мужчина - мужчиной.
==> Train Loss 3.62239715307951 at 65 step


Training:   3%|▎         | 2504/85316 [01:58<1:38:47, 13.97it/s]

Женщина должна быть не только красивой, но и умной.
==> Train Loss 3.7913258901331575 at 70 step


Epochs:  50%|█████     | 1/2 [1:09:47<1:09:47, 4187.95s/it]


KeyboardInterrupt: 

In [11]:
torch.save(model_jokes, 'generation/rularge3.pt')
# model = torch.load('generation/rularge2.pt')

In [6]:
df_val = pd.read_csv('val2.csv')
df_val = df_val[df_val['label'] == 1]
df_val = df_val.reset_index(drop=True)
df_val

,text,label
0,"В такое замечательное утро, как это, особенно ...",1
1,"У тебя не было детства, если ты сам себе не ст...",1
2,"— Ну что ж, у вас отличные рекомендации с пред...",1
3,"Женщины, как грибы: самые красивые - самые ядо...",1
4,"- Это так трогательно! Я аж взбухнул.- Может, ...",1
...,...,...
17094,- Как дела?- Свожу концы с концами.- Работаешь...,1
17095,"Ничто так не возбуждает девушку, как легкие по...",1
17096,"какие ребра, такие и Евы.\n",1
17097,"В фильмах заботливые жёны всегда улыбаются, ко...",1


In [99]:
gen_df.to_csv('jokes_generated.csv', index=False)

In [109]:
gen_df.iloc[7]['text']

'У меня есть мечта, но я не знаю, как ее осуществить.'

In [10]:
df_val.sample(1)['text'].item().split()

['Я,',
 'конечно,',
 'не',
 'идеален,',
 'но',
 'и',
 'ты',
 'не',
 'борщ',
 'со',
 'сметанкой.']

In [11]:
gen_jokes = []

In [14]:
for _ in trange(10000):
    two = df_val.sample(1)['text'].item().split()[:2]
    generated = generate(' '.join(two), model, tokenizer)
    gen_jokes.append(generated)

 25%|██▌       | 2533/10000 [29:28<1:26:54,  1.43it/s]


KeyboardInterrupt: 

In [113]:
generate('Женщина отличается от',model_jokes, tokenizer)

'Женщина отличается от мужчины тем, что она умеет готовить борщ и стирать носки.'

In [3]:
tokenizer2 = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')
model2 = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2').to(device)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
